In [52]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import bids
from bids import BIDSLayout, BIDSValidator
from tqdm import tqdm

import nilearn as nil
from nilearn.input_data import NiftiMasker
from nilearn.image import resample_to_img, resample_img
from nilearn.datasets import load_mni152_template, load_mni152_brain_mask
from nilearn.glm.first_level.hemodynamic_models import compute_regressor
import nibabel as nib

from skimage.measure import block_reduce
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

from scipy.stats import ttest_1samp, ttest_ind, t, sem
import hbayesdm.models

from concurrent.futures import ProcessPoolExecutor

In [2]:
bids.config.set_option('extension_initial_dot', True)

args = {
    'root': '/data2/project_model_based_fmri/ds000005/',
}

In [3]:
def array2pindex(array, p_value=0.05, flatten=False):
    confidence = 1 - p_value
    flattened_array = array.flatten()
    
    n = len(flattened_array)
    m = np.mean(flattened_array)
    std_err = sem(flattened_array)
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    end = m + h
    
    ret = (flattened_array >= end) if flatten is True else (array >= end)
    return ret

In [4]:
def custom_masking(mask_path, p_value, zoom, smoothing_fwhm, interpolation_func, standardize=False, flatten=False):
    if mask_path is None:
        assert (mask_path is None)
    elif type(mask_path) is str:
        mask_files = [mask_path]
    else:
        if type(mask_path) is not type(Path()):
            mask_path = Path(mask_path)
        mask_files = [file for file in mask_path.glob('*.nii.gz')]

    mni152_mask = load_mni152_brain_mask()

    m = array2pindex(nib.load(mask_files[0]).get_fdata(), p_value, flatten)

    if len(mask_files) == 1:
        return m
    else:
        for i in range(1, len(mask_files)):
            m |= array2pindex(nib.load(mask_files[i]).get_fdata(), p_value, flatten)
    
    m = block_reduce(m, zoom, interpolation_func)
    m = 1 * (m > 0)

    m_true = np.array([i for i, v in enumerate(m.flatten()) if v != 0])
    masked_data = nib.Nifti1Image(m, affine=mni152_mask.affine)
    masker = NiftiMasker(mask_img=masked_data,
                         standardize=standardize,
                         smoothing_fwhm=smoothing_fwhm)

    return masked_data, masker, m_true

In [5]:
def image_preprocess(params):
    image_filepath, confounds, motion_confounds, masker, masked_data, num = params
    
    if confounds is not None:
        confounds = pd.read_table(confounds, sep='\t')
        confounds = confounds[motion_confounds]
        confounds = confounds.to_numpy()

    fmri_masked = resample_to_img(image_filepath, masked_data)
    fmri_masked = masker.fit_transform(fmri_masked, confounds=confounds)
    
    return fmri_masked

In [47]:
def bids_preprocess(root,
                    save_path=None,
                    save=True,
                    single_file=False,
                    zoom=(1, 1, 1),
                    smoothing_fwhm=6,
                    interpolation_func=np.mean,
                    motion_confounds=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'],
                    p_value=0.05,
                    task_name='task-zero',
                    standardize=False,
                    ncore=os.cpu_count()):
    
    print('...loading and validating BIDS')
    layout = BIDSLayout(root, derivatives=True)
    nii_layout = layout.derivatives['fMRIPrep'].get(return_type='file', suffix='bold', extension='nii.gz')
    reg_layout = layout.derivatives['fMRIPrep'].get(return_type='file', suffix='regressors', extension='tsv')
    
    n_subject = len(layout.get_subjects())
    n_session = len(layout.get_session())
    n_run = len(layout.get_run())
    print('...done!')
    
    root = Path(root)
    print(layout.derivatives)
    mask_path = Path(layout.derivatives['fMRIPrep'].root) / 'mask'
    print(mask_path)

    print('...make custom mask file')
    masked_data, masker, m_true = custom_masking(mask_path, p_value, zoom, smoothing_fwhm, interpolation_func, standardize)
    print(masked_data.get_fdata().shape)
    print(m_true.shape)
    print('...preprocessing')
    
    print('...image processing using %s cores' % ncore)
    params = [[z[0], z[1], motion_confounds, masker, masked_data, i]
            for i, z in enumerate(zip(nii_layout, reg_layout))]

    with ProcessPoolExecutor() as executor:
        X = np.array(list(executor.map(image_preprocess, params)))
        
        if n_session != 0:
            X = X.reshape(n_subject, n_session, n_run, -1, m_true.shape[0])
        else:
            X = X.reshape(n_subject, n_run, -1, m_true.shape[0])
    print('...done!')

    if save:
        if save_path is None:
            sp = Path(layout.derivatives['fMRIPrep'].root) / 'data'
        else:
            sp = Path(save_path)
            
        if not sp.exists():
            sp.mkdir()
        
        if single_file:
            np.save(sp / 'X.npy', X)
        else:
            for i in range(X.shape[0]):
                np.save(sp / f'X_{i+1}.npy', X[i])
        np.save(sp / 'restore_map.npy', m_true)
    
    return X, masked_data, m_true

In [48]:
import time
s = time.time()
X, masked_data, m_true = bids_preprocess(args['root'], single_file=True, zoom=(2, 2, 2), ncore=os.cpu_count())
e = time.time()
print('time elapsed:', e - s)

...loading and validating BIDS
...done!
{'fMRIPrep': BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48}
/data2/project_model_based_fmri/ds000005/derivatives/fmriprep/mask
...make custom mask file
(46, 55, 46)
(6810,)
...preprocessing
...image processing using 88 cores
...done!
time elapsed: 142.01521801948547


In [49]:
X.shape, m_true.shape

((16, 3, 240, 6810), (6810,))

In [33]:
def preprocess_events(df_events_list, df_events_info):
    new_df_list = df_events_list.copy()
    for i in range(len(new_df_list)):
        new_df_list[i]['run'] = df_events_info[i]['run']
        new_df_list[i]['subjID'] = df_events_info[i]['subject']
        new_df_list[i]['gamble'] = new_df_list[i]['respcat'].apply(lambda x: 1 if x == 1 else 0)
        new_df_list[i]['cert'] = 0 # certain..?
        
    return new_df_list

In [34]:
def calculate_modulation(df_events_list, latent_params):
    new_df_list = df_events_list.copy()
    for i in range(len(new_df_list)):
        idx = new_df_list[i].iloc[0]['subjID']
        new_df_list[i]['rho'] = latent_params.loc[idx]['rho']
        new_df_list[i]['lambda'] = latent_params.loc[idx]['lambda']
        new_df_list[i]['modulation'] = \
            (new_df_list[i]['gain'] ** new_df_list[i]['rho']) \
            - (new_df_list[i]['lambda'] * (new_df_list[i]['loss'] ** new_df_list[i]['rho']))
        
    return new_df_list

In [35]:
funcs = [preprocess_events, calculate_modulation]

In [36]:
def prepare_events(root, funcs, dm_model,
                   hrf_model='glover',
                   save_path=None,
                   save=True,
                   single_file=True,
                   ncore=os.cpu_count()):
    
    layout = BIDSLayout(root, derivatives=True)
    t_r = layout.get_tr()
    events = layout.get(suffix='events', extension='tsv')
    image_sample = nib.load(
        layout.derivatives['fMRIPrep'].get(
            return_type='file',
            suffix='bold',
            extension='nii.gz')[0]
    )
    n_scans = image_sample.shape[-1]
    
    df_events_list = [event.get_df() for event in events]
    df_events_info = [event.get_entities() for event in events]
    
    if len(df_events_list) != len(df_events_info):
        assert()
    
    df_events_list = funcs[0](df_events_list, df_events_info)
        
    dm_model = getattr(hbayesdm.models, dm_model)(
        data=pd.concat(df_events_list), ncore=ncore)

    df_events_list = funcs[1](df_events_list, dm_model.all_ind_pars)
    
    frame_times = t_r * (np.arange(n_scans) + t_r/2)
    
    df_events = pd.concat(df_events_list)
    signals = []
    for name0, group0 in tqdm(df_events.groupby(['subjID'])):
        signal_subject = []
        for name1, group1 in group0.groupby(['run']):
            exp_condition = np.array(group1[['onset', 'duration', 'modulation']]).T

            signal, name = compute_regressor(
                exp_condition=exp_condition,
                hrf_model=hrf_model,
                frame_times=frame_times)
            signal_subject.append(signal)
        
        signal_subject = np.array(signal_subject)
        reshape_target = signal_subject.shape
        
        normalized_signal = minmax_scale(signal_subject.flatten(), axis=0, copy=True)
        normalized_signal = normalized_signal.reshape(-1, n_scans, 1)
        signals.append(normalized_signal)
    signals = np.array(signals)
    
    if save:
        if save_path is None:
            sp = Path(layout.derivatives['fMRIPrep'].root) / 'data'
        else:
            sp = Path(save_path)
            
        if not sp.exists():
            sp.mkdir()
        
        if single_file:
            np.save(sp / 'y.npy', signals)
        else:
            for i in range(signals.shape[0]):
                np.save(sp / f'y_{i+1}.npy', signals[i])
    
    return dm_model, df_events, signals

In [ ]:
a, b, y = prepare_events(args['root'], funcs, dm_model='ra_prospect', ncore=4)

INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl



Model  = ra_prospect
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 4
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 16
 # of (max) trials per subject  = 256

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl


In [ ]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import Sequential, layers, losses, optimizers, datasets
from tensorflow.keras.layers import Dense, BatchNormalization, ReLU
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(X.shape[0])
        self.on_epoch_end()
        
    # for printing the statistics of the function
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        "Denotes the number of batches per epoch"
        
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):  # index : batch no.
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        niis = [X[i] for i  in indexes]
        targets = [y[i] for i in indexes]
        niis = np.array(niis)
        targets = np.array(targets)

        return niis, targets  # return batch

In [ ]:
alpha = 0.001
lambda_par = 1
repeat_N = 100
batch_size = 64
epochs = 100

In [ ]:
X.shape, y.shape

In [ ]:
X = X.reshape(48 * 240, -1)
y = y.reshape(48 * 240, -1)
print(X.shape, y.shape)

In [ ]:
coeffs = []

for repeat_i in range(repeat_N):
    ids = np.arange(X.shape[0])
    train_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=repeat_i)
    train_steps = len(train_ids) // batch_size
    valid_steps = len(test_ids) // batch_size

    X_train = X[train_ids]
    y_train = y[train_ids]
    X_test = X[test_ids]
    y_test = y[test_ids]
    
    train_generator = DataGenerator(X_train, y_train, batch_size, shuffle=True)
    valid_generator = DataGenerator(X_test, y_test, batch_size, shuffle=False)
    
    bst_model_path = f'ml_models/{repeat_i}_best_elasticnet_alpha{alpha:0.3f}_lambda{lambda_par:0.2f}.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True,monitor='val_loss',mode='min',)
    
    model = Sequential()
    model.add(Dense(1, activation='linear', input_shape=(X.shape[-1],),
                    use_bias=True,
                    kernel_regularizer=l1_l2(lambda_par*alpha,lambda_par*(1-alpha)/2),)) 
    model.compile(loss='mse', optimizer='adam',)
    
    model.fit_generator(generator=train_generator, validation_data=valid_generator,
                        steps_per_epoch=train_steps, validation_steps=valid_steps,
                        epochs=epochs,
                        verbose=0, callbacks=[EarlyStopping(monitor='val_loss', patience=5), model_checkpoint])
    model.load_weights(bst_model_path)
        
    pred = model(X_test).numpy()
    mse = ((pred-y_test)**2).mean()
    print(f'INFO [{repeat_i+1}/{repeat_N}] - mse: {mse:.03f}')
    coeff = model.layers[0].get_weights()[0]  
    coeffs.append(coeff)

In [44]:
print('hi')

hi


In [46]:
np.array(coeffs).shape

(100, 6810, 1)

In [53]:
mni = load_mni152_brain_mask()

t_val = ttest_1samp(coeffs,0).statistic

t_map = np.zeros(masked_data.get_fdata().flatten().shape[0])

for i,v in zip(np.nonzero(masked_data.get_fdata().flatten())[0] ,t_val):
    t_map[i] = v
    
t_map = t_map.reshape(masked_data.shape)

t_map = resize(t_map,(91,109,91))

t_map *= mni.get_data()
nii_i = nib.Nifti1Image(t_map, affine=mni.affine)

nii_i.to_filename(f'elasticnet_keras_masked_alpha{alpha:0.3f}_lambda{lambda_par:0.4f}.nii')

print(f'INFO [{alpha}]: done! ')

INFO [0.001]: done! 


/home/mybirth0407/anaconda3/envs/mva/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
